# 1. Blocker Fraud Company
 <font size="2"> Dados e problema negócio hipotéticos</font>
 
A Blocker Fraud Company é uma empresa especializada na detecção de fraudes em transações financeiras feitas atravś de dispositivos móveis.

O modelo de negócio da empresa é do tipo Serviço com a monetização feita por performance do serviço prestado, ou seja, o usuário paga uma taxa fixa sobre o sucesso da detecção de fraude das transações.

## 1.1. Estratégia de expansão

A Blocker Fraud Company está em fase de expansão no Brasil e para adquirir clientes mais rapidamente, ela adotou uma estratégia muito agressiva. A estratégia funciona da seguinte forma:

1. A empresa vai receber 25% do valor de cada transação detectada verdadeiramente como fraude.
2. A empresa vai receber 5% do valor de cada transação detectada como fraude, porém a transação é verdadeiramente legítima.
3. A empresa vai devolver 100% do valor para o cliente, a cada transação detectada como legítima, porém a transação é verdadeiramente uma fraude.

Com essa estratégia agressiva a empresa assume os riscos em falhar na detecção de fraude e é remunerada na detecção assertiva das fraudes.

## 1.2. Objetivo

Criar um modelo de alta precisão e acurácia na detecção de fraudes de transações feitas através de dispositivos móveis.

### 1.2.1. Entregáveis

Modelo em produção no qual seu acesso será feito via API, ou seja, os clientes enviarão suas transações via API para que o seu modelo as classifique como fraudulentas ou legítimas.

Além disso, você precisará entregar um relatório reportando a performance e os resultados do seu modelo em relação ao lucro e prejuízo que a empresa terá ao usar o modelo que você produziu. No seu relatório deve conter as respostas para as seguintes perguntas:

1. Qual a Precisão e Acurácia do modelo?
2. Qual a Confiabilidade do modelo em classificar as transações como legítimas ou fraudulentas?
3. Qual o Faturamento Esperado pela Empresa se classificarmos 100% das transações com o modelo?
4. Qual o Prejuízo Esperado pela Empresa em caso de falha do modelo?
5. Qual o Lucro Esperado pela Blocker Fraud Company ao utilizar o modelo?


# 2. Imports

In [4]:
import numpy as np
import pandas as pd

## 2.1. Utils functions

# 3. Data

## 3.1. Load dataset

In [5]:
df = pd.read_csv('data.csv')

In [3]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


### 3.1.1. Dataset description

In [5]:
df.shape

(6362620, 11)

Vemos que o nosso dataset é composto por 6.362.620 registros e 11 colunas.

In [4]:
df.isna().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

In [15]:
df.describe().applymap(lambda x: f"{x:0.3f}")

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,6362620.000,6362620.000,6362620.000,6362620.000,6362620.000,6362620.000,6362620.000,6362620.000
mean,243.397,179861.904,833883.104,855113.669,1100701.667,1224996.398,0.001,0.000
std,142.332,603858.231,2888242.673,2924048.503,3399180.113,3674128.942,0.036,0.002
min,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,156.000,13389.570,0.000,0.000,0.000,0.000,0.000,0.000
50%,239.000,74871.940,14208.000,0.000,132705.665,214661.440,0.000,0.000
75%,335.000,208721.478,107315.175,144258.410,943036.708,1111909.250,0.000,0.000
max,743.000,92445516.640,59585040.370,49585040.370,356015889.350,356179278.920,1.000,1.000


In [5]:
df.describe(include='O')

,type,nameOrig,nameDest
count,6362620,6362620,6362620
unique,5,6353307,2722362
top,CASH_OUT,C1902386530,C1286084959
freq,2237500,3,113


1. Variáveis númericas
- **step**: representa o tempo no mundo real no qual o registro foi armazenado, cada 1 representa 1 hora.
- **amount**: representa o valor da transação, podemos ver que o intervalo de valores inicia em 0 até aproximadamente 92 milhões, com uma média de aproximadamente 179 mil com a mediana se encontrando em 74 mil.
- **oldbalanceOrg**: é o balanço da conta antes da transação, se encontra em um intervalo de valores entre 0 e aproximadamente 59 milhões, com uma mediana de aproximadamente 14 mil.
- **newbalanceOrig**: o balanço após a transação, se encontra em um intervalo de valores entre 0 e aproximadamente 49 milhões, com uma mediana de 0.
- **oldbalanceDest**: é o destinatário do saldo inicial antes da transação, os usuários que iniciam o nome com M, o valor é zerado, a variável encontra em um intervalo de valores entre 0 e aproximadamente 356 milhões, com uma mediana de aproximadamente 132 mil.
- **newbalanceDest**: é o novo dsetinatário do saldo após a transação, os usuários que iniciam o nome com M, o valor é zerado, a variável encontra em um intervalo de valores entre 0 e aproximadamente 356 milhões, com uma mediana de aproximadamente 132 mil.
- **isFraud**: essa transação foi rotulada como fraude, é uma variável booleana.
- **isFlaggedFraud**: o sistema rotula uma possível fraude quando houver uma tentativa de transferir mais de 200 mil em uma única transação, é uma variável booleana.

2. Variáveis categóricas
- **type**: tipo da transação, temos 5 valores únicos sendo o tipo CASH_OUT o mais frequente, representando aproximadamente 26% de todo os registros.
- **nameOrig**: cliente que iniciou a transação, vemos que temos clientes que fizeram mais de uma transação sendo o mais frequente efetuado 3 transações.
- **nameDest**: cliente que é o destinatário da transação.

**Análises dos valores**

Não temos valores negativos nas transações, porém existe valores negativos podendo indicar:
- possivelmente algum problema em ser nulos 
- verificar se essas transações são válidas

Portanto, vimos que o dataset não possui nulos, fica a ver para os próximos passos lidar com esse tipo de dados zerados nas transações.

### 3.1.2. Verificando os dados de fraude

In [19]:
df.isFraud.value_counts()

0    6354407
1       8213
Name: isFraud, dtype: int64

In [20]:
df.isFlaggedFraud.value_counts()

0    6362604
1         16
Name: isFlaggedFraud, dtype: int64

In [23]:
(16/(6362604+16))*100

0.000251468734577894

**Análises dos valores**

Nosso dataset é extremamente não balanceado, somente 0.13% foram transações rotuladas como fraude, isso se segue para aquelas que o próprio sistema configurou como fraude, somente 0.00025%.

# 4. Defining traning, testing and validation dataset

In [9]:
df_validation = df[df["step"].isin([740,741,742,743])].copy()
df_train = df[~df["step"].isin([740,741,742,743])].copy()

In [11]:
df_validation.to_parquet('df_validation.parquet.gzip',
              compression='gzip')

df_train.to_parquet('df_train.parquet.gzip',
              compression='gzip')

# 5. Conclusão

Fizemos uma análise superficial acerca dos dados recebidos do cliente, entendemos a dimensão dos nossos dados, verificamos se existe a presença de campos nulos que não foi encontrado e partimos para analisar os valores contidos nas variáveis para ajudar a entender a descrição de cada campo, com isso tempos que:
- Nos campos de transações existem valores zerados, com isso abriu a dúvida se esse valor está correto ou podemos considerar como nulo
- Existem variáveis categóricas bem frequentes e além disso temos clientes considerados Merchants que não possuímos informações sobre o balanço dele.

Concluimos que a próxima etapa devemos executar uma análise exploratória mais aprofundada para que possamos entender os padrões sobre os nossos dados, com isso podemos retirar insights que venha a ajudar a modelar os dados para o nosso problema de negócio.